# 🧱 Lab: Hello LLM - Your First AI App

**Module 1: Setup & Working Style for LLM Apps** | **Duration: ~30 min** | **Type: Lab (Wall)**

---

## Learning Objectives

By the end of this lab, you will have built a working LLM application that:

1. **Loads** API keys securely from environment variables
2. **Switches** between cloud (OpenAI) and local (Ollama) models
3. **Controls** determinism with temperature and seed
4. **Logs** interactions for debugging

## Prerequisites (Concepts Covered)

| Concept | From |
|---------|------|
| API Keys & Environment Variables | mini-env-setup |
| Secrets Hygiene | mini-env-setup |
| Open vs Closed Models | mini-ollama-setup |
| Local LLMs & Ollama | mini-ollama-setup |
| Determinism Controls | This lab |
| Basic Logging | This lab |

## 1. Project Setup

First, let's set up our project structure using uv:

```bash
# Create and navigate to project
uv init hello-llm && cd hello-llm

# Add dependencies
uv add openai python-dotenv
```

Your `.env` file should have:
```bash
OPENAI_API_KEY=sk-your-key-here
```

In [ ]:
import os
import logging
from datetime import datetime
from typing import Literal
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

print("✓ Imports loaded")

## 2. Basic Logging Setup

Logging helps debug LLM applications - track inputs, outputs, and errors.

**Key Rule:** Never log API keys or sensitive data!

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger("hello-llm")

# Test logging
logger.info("Logging configured")
logger.debug("This won't show (level is INFO)")
logger.warning("This is a warning")

## 3. Secure Configuration

Create a configuration class that safely loads and validates settings.

In [ ]:
class Config:
    """Application configuration - loads from environment."""
    
    def __init__(self):
        self.openai_key = os.getenv("OPENAI_API_KEY")
        self.default_model = os.getenv("DEFAULT_MODEL", "gpt-4o-mini")
        self.ollama_url = os.getenv("OLLAMA_URL", "http://localhost:11434/v1")
        
        logger.info("Config loaded from environment")
        # NEVER log actual keys!
        logger.info(f"OpenAI key: {'configured' if self.openai_key else 'missing'}")
    
    def validate(self) -> bool:
        """Validate required configuration."""
        if not self.openai_key:
            logger.error("OPENAI_API_KEY not set")
            return False
        return True

config = Config()
config.validate()

## 4. Determinism Controls

LLMs are probabilistic - same input can give different outputs. Control this with:

| Parameter | Effect |
|-----------|--------|
| `temperature=0` | Most deterministic (greedy) |
| `seed=123` | Reproducible randomness |
| `top_p=1.0` | Full probability distribution |

In [ ]:
# Demonstrate determinism with temperature=0
client = OpenAI()

def deterministic_call(prompt: str, seed: int = 42):
    """Make a deterministic API call."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # Most deterministic
        seed=seed,      # Reproducible
        max_tokens=50
    )
    return response.choices[0].message.content

# Run same prompt twice - should get identical results
prompt = "What is 2+2? Answer with just the number."

result1 = deterministic_call(prompt)
result2 = deterministic_call(prompt)

print(f"First call:  {result1}")
print(f"Second call: {result2}")
print(f"Identical: {result1 == result2}")

## 5. Multi-Provider LLM Client

Build a client that works with both OpenAI and Ollama.

In [ ]:
import requests

def list_ollama_models() -> list[str]:
    """Get available Ollama models."""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=3)
        if response.status_code == 200:
            return [m["name"] for m in response.json().get("models", [])]
    except:
        pass
    return []


class LLMClient:
    """Unified LLM client for multiple providers."""
    
    def __init__(self, provider: Literal["openai", "ollama"] = "openai"):
        self.provider = provider
        
        if provider == "openai":
            self.client = OpenAI()
            self.model = "gpt-4o-mini"
        else:
            self.client = OpenAI(
                base_url="http://localhost:11434/v1",
                api_key="ollama"
            )
            models = list_ollama_models()
            self.model = models[0] if models else "llama3.2:3b"
        
        logger.info(f"LLMClient initialized: {provider}/{self.model}")
    
    def chat(self, message: str, temperature: float = 0.7, seed: int = None) -> str:
        """Send a chat message and get a response."""
        
        logger.info(f"Chat request: {message[:50]}...")
        
        kwargs = {
            "model": self.model,
            "messages": [{"role": "user", "content": message}],
            "temperature": temperature,
            "max_tokens": 200
        }
        
        if seed is not None:
            kwargs["seed"] = seed
        
        response = self.client.chat.completions.create(**kwargs)
        result = response.choices[0].message.content
        
        logger.info(f"Response received: {len(result)} chars")
        return result

# Test with OpenAI
llm = LLMClient("openai")
print(f"\n{llm.chat('Hello! Say hi in one sentence.')}")

In [ ]:
# Test with Ollama (if available)
if list_ollama_models():
    llm_local = LLMClient("ollama")
    print(f"\n{llm_local.chat('Hello! Say hi in one sentence.')}")
else:
    print("⚠️ Ollama not available - skipping local test")

## 6. Complete Application

Let's bring everything together into a simple Q&A assistant.

In [ ]:
from IPython.display import display, Markdown

def md(text):
    """Display text as rendered markdown."""
    display(Markdown(text))


class HelloLLM:
    """A simple Q&A assistant demonstrating all setup concepts."""
    
    def __init__(self, provider: str = "openai", deterministic: bool = False):
        self.llm = LLMClient(provider)
        self.deterministic = deterministic
        self.history = []
        
        logger.info(f"HelloLLM started: provider={provider}, deterministic={deterministic}")
    
    def ask(self, question: str) -> str:
        """Ask a question and get an answer."""
        
        # Set determinism parameters
        temperature = 0 if self.deterministic else 0.7
        seed = 42 if self.deterministic else None
        
        # Get response
        answer = self.llm.chat(question, temperature=temperature, seed=seed)
        
        # Log interaction (without sensitive data)
        self.history.append({
            "timestamp": datetime.now().isoformat(),
            "question": question,
            "answer_length": len(answer),
            "provider": self.llm.provider,
            "model": self.llm.model
        })
        
        return answer
    
    def show_history(self):
        """Display interaction history."""
        print(f"\n📜 History ({len(self.history)} interactions):")
        for i, h in enumerate(self.history, 1):
            print(f"  {i}. [{h['timestamp'][:19]}] {h['question'][:40]}... ({h['answer_length']} chars)")

In [ ]:
# Create assistant with OpenAI
assistant = HelloLLM(provider="openai", deterministic=True)

# Ask some questions
questions = [
    "What is machine learning in one sentence?",
    "Name 3 popular programming languages.",
    "What does API stand for?"
]

for q in questions:
    print(f"\n❓ {q}")
    answer = assistant.ask(q)
    md(f"**Answer:** {answer}")

In [ ]:
# Show interaction history
assistant.show_history()

## 7. Comparing Providers

Let's compare responses from different providers on the same question.

In [ ]:
def compare_providers(question: str):
    """Compare responses from different providers."""
    
    print(f"\n❓ Question: {question}")
    print("=" * 60)
    
    # OpenAI
    openai_llm = LLMClient("openai")
    openai_response = openai_llm.chat(question, temperature=0)
    print(f"\n☁️ OpenAI ({openai_llm.model}):")
    print(f"   {openai_response}")
    
    # Ollama (if available)
    if list_ollama_models():
        ollama_llm = LLMClient("ollama")
        ollama_response = ollama_llm.chat(question, temperature=0)
        print(f"\n💻 Ollama ({ollama_llm.model}):")
        print(f"   {ollama_response}")
    else:
        print("\n💻 Ollama: Not available")

compare_providers("Explain recursion in one sentence.")

## 🎯 Summary

### What We Built

A complete LLM application with:

1. **Secure Configuration** - API keys loaded from environment
2. **Multi-Provider Support** - Works with OpenAI and Ollama
3. **Determinism Controls** - Reproducible outputs with temperature=0 and seed
4. **Logging** - Track interactions without exposing secrets

### Key Patterns

```python
# Secure config loading
load_dotenv()
client = OpenAI()  # Auto-reads OPENAI_API_KEY

# Deterministic calls
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[...],
    temperature=0,
    seed=42
)

# Switch to local model
local_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)
```

### Checklist

- [x] Environment variables configured
- [x] Secrets hygiene followed
- [x] OpenAI and Ollama both working
- [x] Determinism controls understood
- [x] Basic logging implemented

### Next Steps

You're now ready to explore **Module 2: LLM Core Concepts** where you'll learn about:
- Tokenization and context windows
- Temperature and sampling parameters
- Streaming responses